In [1]:
import pandas as pd
import sys
from datetime import datetime
sys.path.append('..')

In [2]:
data_train = pd.read_csv("../data/data_train.csv")
data_val = pd.read_csv("../data/data_val.csv")
data_test = pd.read_csv("../data/data_test.csv")

In [3]:
data_train["date"] = data_train["date"].astype("datetime64[ns]")
data_val["date"] = data_val["date"].astype("datetime64[ns]")
data_test["date"] = data_test["date"].astype("datetime64[ns]")

In [4]:
data_train = data_train.groupby("date").agg({
    **{
        "prepaid_amount": "sum",
        "years_to_matur": "median",
        "age_owner_years": "median",
        "age_loan_years": "median",
        "volume_schedule": "sum",
        "FIXED_MONTHLY_EXPENSES": "sum",
        "INCOME_houshold": "sum",
        "dpd": "sum"
    }
    ,**{key: "max"
    for key in [
        'avg_monthly_product_client_rate_cln',
        'avg_monthly_product_client_mtg',
        'avg_monthly_product_client_rate_mtg_grn', 'avg_empl_enterprise',
        'register_unemployed', 'unemployment_rate',
        'avg_monthly_salary_enterprise_val',
        'avg_monthly_salary_enterprise_index', 'wheat_purchase_price_index',
        'milk_purchase_price_index', 'production_price_energy_index',
        'production_price_water_supply_index', 'inflation',
        'inflation_apartment_usage', 'new_flats', 'economy_index',
        'economy_index_real_estate'
        ]
}})

In [5]:
data_val = data_val.groupby("date").agg({
    **{
        "prepaid_amount": "sum",
        "years_to_matur": "median",
        "age_owner_years": "median",
        "age_loan_years": "median",
        "volume_schedule": "sum",
        "FIXED_MONTHLY_EXPENSES": "sum",
        "INCOME_houshold": "sum",
        "dpd": "sum"
    }
    ,**{key: "max"
    for key in [
        'avg_monthly_product_client_rate_cln',
        'avg_monthly_product_client_mtg',
        'avg_monthly_product_client_rate_mtg_grn', 'avg_empl_enterprise',
        'register_unemployed', 'unemployment_rate',
        'avg_monthly_salary_enterprise_val',
        'avg_monthly_salary_enterprise_index', 'wheat_purchase_price_index',
        'milk_purchase_price_index', 'production_price_energy_index',
        'production_price_water_supply_index', 'inflation',
        'inflation_apartment_usage', 'new_flats', 'economy_index',
        'economy_index_real_estate'
        ]
}})

In [6]:
data_test = data_test.groupby("date").agg({
    **{
        "prepaid_amount": "sum",
        "years_to_matur": "median",
        "age_owner_years": "median",
        "age_loan_years": "median",
        "volume_schedule": "sum",
        "FIXED_MONTHLY_EXPENSES": "sum",
        "INCOME_houshold": "sum",
        "dpd": "sum"
    }
    ,**{key: "max"
    for key in [
        'avg_monthly_product_client_rate_cln',
        'avg_monthly_product_client_mtg',
        'avg_monthly_product_client_rate_mtg_grn', 'avg_empl_enterprise',
        'register_unemployed', 'unemployment_rate',
        'avg_monthly_salary_enterprise_val',
        'avg_monthly_salary_enterprise_index', 'wheat_purchase_price_index',
        'milk_purchase_price_index', 'production_price_energy_index',
        'production_price_water_supply_index', 'inflation',
        'inflation_apartment_usage', 'new_flats', 'economy_index',
        'economy_index_real_estate'
        ]
}})

In [7]:
mins = {}
maxes = {}
for column in data_test.columns:
    maxes[column] = data_train[column].max()
    mins[column] = data_train[column].min()
    data_train[column] = (data_train[column] - mins[column])/ (maxes[column] - mins[column])
    data_val[column]   = (data_val[column]   - mins[column])/ (maxes[column] - mins[column])
    data_test[column]  = (data_test[column]  - mins[column])/ (maxes[column] - mins[column])

In [8]:
data_train = data_train.append(data_val)
del data_train["avg_monthly_product_client_rate_mtg_grn"]

In [9]:
y_train = data_train["prepaid_amount"]
del data_train["prepaid_amount"]

In [10]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from matplotlib import pyplot
import numpy as np
 
# feature selection
def select_features(X_train, y_train, n_features=None):
	# configure to select all features
	fs = SelectKBest(score_func=f_regression, k='all' if n_features is None else n_features)
	# learn relationship from training data
	fs.fit(X_train, y_train)
	return fs
 

In [11]:
n_features = 5
dependent_variable = "prepaid_amount"
# feature selection
fs = select_features(data_train, y_train, n_features)
# what are scores for the features
best_features = data_train.columns[np.argsort(fs.scores_)[-n_features:]]

data_train = data_train[best_features]
data_train[dependent_variable] = y_train
data_test = data_test[list(best_features) + [dependent_variable]]

# Best models

In [12]:
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from fbprophet import Prophet

In [13]:
def evaluate(y_true, y_pred):
    y_true = np.squeeze(y_true)
    y_pred = np.squeeze(y_pred)
    scores = {}
    scores["mae"] = mean_absolute_error(y_true, y_pred)
    scores["mse"] = mean_squared_error(y_true, y_pred)
    return scores


In [14]:
def sarimax(
        train,
        val,
        col,
        exog_col=None,
        freq='M',
        eval_fun=evaluate,
        p=1,
        d=0,
        q=1,
        trend="c",
        P=0,
        D=0,
        Q=0,
        use_boxcox=True,
):
    train_data = train.copy()
    val_data = val.copy()

    xreg = None
    xreg_fcst = None
    if exog_col is not None:
        xreg = train_data[list(exog_col)]
        xreg_fcst = val_data[list(exog_col)]

    s = freq_convert(freq)

    mod = sm.tsa.statespace.SARIMAX(
        endog=train_data[col],
        exog=xreg,
        order=(p, d, q),
        seasonal_order=(P, D, Q, s),
        trend=trend,
        use_boxcox=use_boxcox,
        initialization_method="estimated",
        freq=freq
    )
    res = mod.fit(method='powell', maxiter=1000, disp=False)
    f = res.fittedvalues

    forecast = res.forecast(steps=val_data.shape[0], exog=xreg_fcst)
    return eval_fun(val_data[col], forecast), pd.concat((f, forecast))

In [15]:
 def freq_convert(freq_str):
    if freq_str == 'M' or freq_str == 'MS':
        return 12
    elif freq_str == 'W':
        return 7
    elif freq_str == 'D':
        return None  # TODO check it
    else:
        raise AttributeError


def prepare_data_prophet(train, val, col, exog_col=None):
    colnames = [col]
    if exog_col is not None:
        colnames += exog_col

    train_data = train[colnames].copy().rename(columns={col: 'y'})
    train_data["ds"] = train.index

    val_data = val[colnames].copy().rename(columns={col: 'y'})
    val_data["ds"] = val.index

    return train_data, val_data

def prophet(train, val, col, exog_col=None, freq='MS', eval_fun=evaluate):
    train_data, val_data = prepare_data_prophet(train, val, col, exog_col)

    m = Prophet()
    if exog_col is not None:
        for c in exog_col:
            m.add_regressor(name=c)

    m.fit(train_data)

    future = m.make_future_dataframe(periods=val_data.shape[0], freq=freq)
    if exog_col is not None:
        for c in exog_col:
            future[c] = np.concatenate(
                (train_data[c].to_numpy(), val_data[c].to_numpy())
            )

    forecast = m.predict(future)
    f = forecast.tail(val_data.shape[0])["yhat"].values
    in_sample = m.predict()
    return eval_fun(val_data["y"].values, f), forecast["yhat"].values

In [16]:
def ets(
        train,
        val,
        col="org",
        exog_col=None,
        freq='M',
        eval_fun=evaluate,
        error='add',
        trend='add',
        damped_trend=False,
        seasonal=None,
        epsilon=0
):
    train_data = train[col].copy() + epsilon
    val_data = val[col].copy() + epsilon

    seasonal_periods = freq_convert(freq)

    mod = ETSModel(
        train_data,
        error=error,
        trend=trend,
        damped_trend=damped_trend,
        seasonal=seasonal,
        seasonal_periods=seasonal_periods,
        freq=freq,
        initialization_method="estimated",
    )
    res = mod.fit()
    f = res.fittedvalues

    forecast = res.forecast(steps=val_data.shape[0]) - epsilon
    return eval_fun(val_data, forecast), pd.concat((f, forecast))

In [17]:
results_prophet = prophet(train=data_train,   
    val=data_test, 
    col="prepaid_amount", 
    exog_col=None)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [18]:
results_ets = ets(train=data_train,   
    val=data_test, 
    col="prepaid_amount", 
    exog_col=None)

In [19]:
results_sarimax = sarimax(train=data_train,   
    val=data_test, 
    col="prepaid_amount", 
    exog_col=[c for c in data_train.columns if c!=dependent_variable])

/Users/mdolinski/.local/share/virtualenvs/INGModellingContest-QFb3_eOW/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  warnings.warn('No frequency information was'


In [20]:
final_predictions = ((results_prophet[1] + results_ets[1] + results_sarimax[1]) / 3) * (maxes[dependent_variable] - mins[dependent_variable]) + mins[dependent_variable]

In [21]:
final_predictions

2016-01-31    2.114380e+06
2016-02-29    2.182795e+06
2016-03-31    1.999452e+06
2016-04-30    2.785902e+06
2016-05-31    2.750598e+06
2016-06-30    2.569749e+06
2016-07-31    3.045862e+06
2016-08-31    3.297355e+06
2016-09-30    3.547296e+06
2016-10-31    3.525413e+06
2016-11-30    3.493623e+06
2016-12-31    3.846221e+06
2017-01-31    3.682191e+06
2017-02-28    3.868804e+06
2017-03-31    4.095995e+06
2017-04-30    4.355657e+06
2017-05-31    4.528070e+06
2017-06-30    4.442708e+06
2017-07-31    4.561724e+06
2017-08-31    5.470940e+06
2017-09-30    6.406939e+06
2017-10-31    7.152401e+06
2017-11-30    7.129197e+06
2017-12-31    8.159417e+06
2018-01-31    7.991780e+06
2018-02-28    7.725202e+06
2018-03-31    8.225417e+06
2018-04-30    8.099157e+06
2018-05-31    8.656614e+06
2018-06-30    8.172766e+06
2018-07-31    7.910565e+06
2018-08-31    8.200874e+06
2018-09-30    8.845887e+06
2018-10-31    9.442885e+06
2018-11-30    8.911650e+06
2018-12-31    9.000193e+06
2019-01-31    9.077278e+06
2